In [ ]:
#TASK 2 Solution:

In [ ]:
#Load necessary packages

import pandas as pd
#import numpy as np
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import os
from numpy import array
import numpy as np
import torch
from numpy import argmax
import torch.nn as nn
import pickle


In [ ]:
torch.cuda.is_available()
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"

In [ ]:
#Load the data

data = pd.read_csv(r'bengali_hatespeech.csv')
data

In [ ]:
#: Splitting off a part of the Bengali corpus such that it roughly
#equals the Hindi corpus in size and distribution of classes (hatespeech/non-hatespeech)

In [ ]:
#Includes positive and negative(non-hatespeech/hatespeech) data frame sets

first_set_data= data.head(100)
last_set_data=data.tail(100)

In [ ]:
#Concatenating the data frames into resultant single set of data frames with both positive and negative classes

frames=[first_set_data, last_set_data]
bengali_data = pd.concat(frames)
bengali_data

In [ ]:
#Preprocessing the data to lower text, remove punctuations if any and remove the emoji or emoticons

bengali_data['sentence']= bengali_data['sentence'].str.lower()
bengali_data['sentence'] = bengali_data['sentence'].str.replace('[!”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]','')
bengali_data['sentence'] = bengali_data['sentence'].str.replace('[^\w\s#@/:%.,_-]', '', flags=re.UNICODE)

In [ ]:
#Displaying few data frames with columns of sentence as text and hate as class one(positive) and zero(negative)

bengali_data.head()
bengali_data.tail()

In [ ]:
#List of all Bengali words

vocab = bengali_data['sentence'].values.tolist()
vocab= [words for lines in vocab for words in lines.split()]


In [ ]:
#Calculating the probability to keep words in context

bengali_freq = {} 
bengali_relative_freq={}
p_keep_bengali={}
for item in vocab: 
    if (item in bengali_freq): 
        bengali_freq[item] += 1
    else: 
        bengali_freq[item] = 1


for item in bengali_freq:
    bengali_relative_freq[item]=bengali_freq[item]/len(vocab)
    
        
        
for x, y in bengali_relative_freq.items():
    p_keep_bengali[x]=(np.sqrt(y/0.001) +1) * (0.001/y) 



print(p_keep_bengali)

In [ ]:
#Calculating one hot vector

def word_to_one_hot(word):
    one_hot_vector=np.zeros((len(bengali_freq)))
    
    index=list(bengali_freq.keys()).index(word)
    one_hot_vector[index]=1
    
    return one_hot_vector
       


#---test data--#
#corresponding_one_hot_encoded=word_to_one_hot('যততসব')

#print(corresponding_one_hot_encoded)    
    

In [ ]:
file = open("bengali_freq_dictionary.txt", "wb")
pickle.dump(bengali_freq, file)
file.close()

#bengali_freq

In [ ]:
def sampling_prob(word):
    return p_keep[word]

#--test--#
#print(sampling_prob('যততসব'))
#----

In [ ]:
# Set hyperparameters
window_size = 5
embedding_size = 300

In [ ]:
def get_target_context_helper(variable_sentence,current_index):
    #takes in the new sentence and the curresponding index
    
    word_list=[]
    i=current_index
    j=current_index
    lower_limit=current_index-window_size
    upper_limit=current_index+window_size

    if lower_limit<0:
        lower_limit=0
    if upper_limit> len(variable_sentence)-1:
        upper_limit=len(variable_sentence)-1

    while(i>lower_limit):
        word_list.append(variable_sentence[i-1])
        i=i-1

    while (j<upper_limit):
        word_list.append(variable_sentence[j+1])
        j=j+1
    
    return variable_sentence[current_index], word_list
    #returns the currespong word and the list of words in its context

def get_target_context(sentence):
    
    sentence=sentence.split()
    new_sentence=[]
    for each_word in sentence:
        float_value=float(p_keep_bengali[each_word])
        if np.random.random() < float_value:
            new_sentence.append(each_word)
    
    p=[]
   
    len_new=len(new_sentence)
    for i in range (len_new):
        #print(get_target_context_helper(new_sentence,i)) #passes the new sentences and each index to do the window
        p.append(get_target_context_helper(new_sentence,i))

        
    return p


each_sentence=bengali_data['sentence']
list_of_all_words_and_corresponding_context_words=[]
for every_sentence in each_sentence:
    list_of_all_words_and_corresponding_context_words.append(get_target_context(every_sentence))
    
list_of_all_words_and_corresponding_context_words = [item for sublist in list_of_all_words_and_corresponding_context_words for item in sublist]
print(list_of_all_words_and_corresponding_context_words)
#returns the list of all the focus and context words for the entire vocabulary

In [ ]:
# More hyperparameters
learning_rate = 0.025
epochs = 100

In [ ]:
# Create model 

class Word2Vec(nn.Module):
  def __init__(self,embedding_size,Vocabulary_size):
    super(Word2Vec,self).__init__()
    
    self.l1=nn.Linear(Vocabulary_size,embedding_size)
    self.l2=nn.Linear(embedding_size,Vocabulary_size)
    self.softmax=nn.Softmax(dim=1)


  def forward(self, one_hot):
    out=self.l1(one_hot)
    out=self.l2(out)
    out=self.softmax(out)
    return out


model=Word2Vec(embedding_size,len(bengali_freq))


In [ ]:
#onehot to torch transform

def transform_to_torch(corresponding_one_hot_encoded):
    corresponding_one_hot_encoded_torch = torch.from_numpy(corresponding_one_hot_encoded.astype(np.float32))
    corresponding_one_hot_encoded_torch = corresponding_one_hot_encoded_torch.view(corresponding_one_hot_encoded_torch.shape[0],1)
    corresponding_one_hot_encoded_torch=corresponding_one_hot_encoded_torch.t()
    return corresponding_one_hot_encoded_torch

In [ ]:
# Define optimizer and loss
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
criterion = nn.NLLLoss()


In [ ]:
# Define train procedure

# load initial weights

n_iters=100

def train():
    for epoch in range(n_iters):
        
        for each_word in list_of_all_words_and_corresponding_context_words:
            focus,context_words=each_word
            #print("focus is",focus)
            focus =word_to_one_hot(focus)
            focus =transform_to_torch(focus)
            
            
            for each_context_word in context_words:
                #print("contexts are",each_context_word)
                each_context_word=word_to_one_hot(each_context_word)
                each_context_word =transform_to_torch(each_context_word)
                Y_pred=model(focus)
                #torch.max(labels, 1)[1]
                loss=criterion(Y_pred,torch.max(each_context_word, 1)[1])
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                
        print("loss is",loss)

#train() #uncomment the train function to run for 100 epochs.


print("Training finished")

In [ ]:
#save the model as bengali.pth

model_path = F"bengali.pth"
#torch.save(model.state_dict(), model_path)

##for implementation representation the data was run on 200 rows. The code for the entire data set was run in collab and saved
#bengali.pth